# Entrenamiento de Modelos con MLFlow

## Objetivo
Entrenar y comparar modelos de regresión para predecir precios de automóviles, utilizando MLFlow para:
1. Registrar experimentos y parámetros
2. Rastrear métricas de desempeño
3. Guardar y versionar modelos
4. Facilitar la reproducibilidad

## Modelos a entrenar:
- **Regresión Lineal**: Modelo base simple
- **Random Forest**: Modelo más complejo con mejor capacidad de capturar relaciones no lineales

In [51]:
# Importar librerías
import pandas as pd
import numpy as np
from pathlib import Path
import joblib

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import lightgbm as lgb

# MLFlow para tracking de experimentos
import mlflow
import mlflow.sklearn

# Configuración
import warnings
warnings.filterwarnings('ignore')

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


## 1. Configuración de MLFlow

MLFlow es una plataforma para gestionar el ciclo de vida de Machine Learning, incluyendo experimentación, reproducibilidad y deployment.

In [52]:
# Configurar MLFlow
import os

# Detectar automáticamente la raíz del proyecto
# Si estamos en notebooks/, subir un nivel para llegar a la raíz del proyecto
current_dir = Path.cwd()
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
else:
    project_root = current_dir

# Directorio MLFlow SIEMPRE en la raíz del proyecto (relativo)
mlflow_dir = project_root / 'mlruns'
mlflow_dir.mkdir(exist_ok=True)

# Establecer URI de tracking
mlflow.set_tracking_uri(f"file:///{mlflow_dir.absolute()}")

# Crear o usar experimento existente
experiment_name = "prediccion_precios_automoviles"
mlflow.set_experiment(experiment_name)

print(f"✅ MLFlow configurado correctamente")
print(f"📁 Proyecto: {project_root}")
print(f"📊 MLFlow: {mlflow_dir.relative_to(project_root)}")
print(f"🔬 Experimento: {experiment_name}")
print(f"\n{'='*70}")
print("💡 Para visualizar experimentos en MLFlow UI:")
print(f"{'='*70}")
print(f"")
print(f"1️⃣  Navega a la raíz del proyecto:")
print(f"   cd {project_root}")
print(f"")
print(f"2️⃣  Ejecuta el script (RECOMENDADO):")
print(f"   ./start_mlflow.sh")
print(f"")
print(f"   O manualmente:")
print(f"   mlflow ui")
print(f"")
print(f"3️⃣  Abre en navegador: http://localhost:5000")
print(f"{'='*70}")

✅ MLFlow configurado correctamente
📁 Proyecto: /Users/david.palacio/Documents/academia/data-projects-lab/projects/prediccion-precios-automoviles
📊 MLFlow: mlruns
🔬 Experimento: prediccion_precios_automoviles

💡 Para visualizar experimentos en MLFlow UI:

1️⃣  Navega a la raíz del proyecto:
   cd /Users/david.palacio/Documents/academia/data-projects-lab/projects/prediccion-precios-automoviles

2️⃣  Ejecuta el script (RECOMENDADO):
   ./start_mlflow.sh

   O manualmente:
   mlflow ui

3️⃣  Abre en navegador: http://localhost:5000


## 2. Carga y Preparación de Datos

In [53]:
# Cargar dataset
data_path = Path('../data/raw/automoviles_usados.parquet')
df = pd.read_parquet(data_path)

print(f"Dataset cargado: {len(df)} registros")
df.head()

Dataset cargado: 10000 registros


,marca,tipo_carroceria,año,kilometraje,tipo_combustible,transmision,cilindrada,potencia,peso,consumo,color,edad_propietarios,calificacion_estado,region_venta,precio
0,Ford,Hatchback,2015,400000,Gasolina,Automática,2039,93,1690,7.269729,Rojo,1,7.269855,Sur,2400.0
1,Mercedes-Benz,Sedán,2021,400000,Gasolina,Manual,1606,91,1476,7.508579,Gris,1,7.224478,Sur,15500.0
2,Hyundai,Hatchback,2015,400000,Gasolina,Manual,2204,120,1373,8.237119,Azul,2,7.377436,Oeste,2100.0
3,Nissan,Sedán,2017,400000,Diesel,Automática,1894,96,1191,6.526941,Blanco,4,4.944675,Norte,2600.0
4,Honda,Hatchback,2020,400000,Gasolina,Automática,1775,80,1139,7.587182,Azul,2,3.025227,Sur,2200.0


In [54]:
# Separar características (X) y variable objetivo (y)
X = df.drop('precio', axis=1)
y = df['precio']

print(f"Características (X): {X.shape}")
print(f"Variable objetivo (y): {y.shape}")

Características (X): (10000, 14)
Variable objetivo (y): (10000,)


In [55]:
# Identificar columnas numéricas y categóricas
# Excluimos 'precio' ya que es la variable objetivo
numeric_features = ['año', 'kilometraje', 'cilindrada', 'potencia', 'peso', 
                   'consumo', 'edad_propietarios', 'calificacion_estado']

categorical_features = ['marca', 'tipo_carroceria', 'tipo_combustible', 
                       'transmision', 'color', 'region_venta']

print(f"Variables numéricas: {len(numeric_features)}")
print(numeric_features)
print(f"\nVariables categóricas: {len(categorical_features)}")
print(categorical_features)

Variables numéricas: 8
['año', 'kilometraje', 'cilindrada', 'potencia', 'peso', 'consumo', 'edad_propietarios', 'calificacion_estado']

Variables categóricas: 6
['marca', 'tipo_carroceria', 'tipo_combustible', 'transmision', 'color', 'region_venta']


## 3. División Train/Test

Dividimos los datos en conjuntos de entrenamiento (80%) y prueba (20%) para evaluar el desempeño del modelo en datos no vistos.

In [56]:
# Split train/test con semilla fija para reproducibilidad
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

print(f"Conjunto de entrenamiento: {X_train.shape[0]} registros")
print(f"Conjunto de prueba: {X_test.shape[0]} registros")
print(f"\nProporción train/test: {X_train.shape[0]/len(df):.1%} / {X_test.shape[0]/len(df):.1%}")

Conjunto de entrenamiento: 8000 registros
Conjunto de prueba: 2000 registros

Proporción train/test: 80.0% / 20.0%


## 4. Pipeline de Preprocesamiento

Creamos un pipeline que:
- **Variables numéricas**: Aplica StandardScaler para normalizar (media=0, std=1)
- **Variables categóricas**: Aplica OneHotEncoder para convertir categorías en variables binarias

El uso de pipelines asegura que el preprocesamiento sea consistente entre train y test.

In [57]:
# Crear transformadores para cada tipo de variable
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Combinar transformadores en un ColumnTransformer
# Esto permite aplicar diferentes transformaciones a diferentes columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

print("Pipeline de preprocesamiento creado.")

Pipeline de preprocesamiento creado.


## 5. Función para Evaluar Modelos

Creamos una función para calcular métricas estándar de regresión:
- **RMSE** (Root Mean Squared Error): Error cuadrático medio, penaliza errores grandes
- **MAE** (Mean Absolute Error): Error absoluto medio, más robusto a outliers
- **R²** (Coeficiente de determinación): Proporción de varianza explicada (0-1, mayor es mejor)

In [58]:
def evaluate_model(y_true, y_pred, model_name="Modelo"):
    """
    Calcula y muestra métricas de evaluación para un modelo de regresión.
    
    Args:
        y_true: Valores reales
        y_pred: Valores predichos
        model_name: Nombre del modelo para identificación
    
    Returns:
        dict: Diccionario con las métricas calculadas
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred)) # sqrt para traer la raiz cuadrada
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    metrics = {
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    }
    
    print(f"\n{'='*60}")
    print(f"Métricas para {model_name}:")
    print(f"{'='*60}")
    print(f"  RMSE (Root Mean Squared Error): ${rmse:,.2f}")
    print(f"  MAE  (Mean Absolute Error):     ${mae:,.2f}")
    print(f"  R²   (R-squared):               {r2:.4f} ({r2*100:.2f}%)")
    print(f"{'='*60}\n")
    
    return metrics

## 6. Entrenamiento - Modelo 1: Regresión Lineal

La regresión lineal es un modelo simple que asume relaciones lineales entre características y la variable objetivo. Sirve como baseline para comparar modelos más complejos.

In [59]:
# Iniciar un run de MLFlow para este experimento
with mlflow.start_run(run_name="linear_regression"):
    
    print("Entrenando modelo: Regresión Lineal")
    
    # Crear pipeline completo: preprocesamiento + modelo
    pipeline_lr = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ])
    
    # Entrenar modelo
    pipeline_lr.fit(X_train, y_train)
    print("Modelo entrenado.")
    
    # Hacer predicciones
    y_train_pred = pipeline_lr.predict(X_train)
    y_test_pred = pipeline_lr.predict(X_test)
    
    # Evaluar en conjunto de entrenamiento
    print("\nEvaluación en conjunto de ENTRENAMIENTO:")
    train_metrics = evaluate_model(y_train, y_train_pred, "Regresión Lineal - Train")
    
    # Evaluar en conjunto de prueba
    print("Evaluación en conjunto de PRUEBA:")
    test_metrics = evaluate_model(y_test, y_test_pred, "Regresión Lineal - Test")
    
    # Registrar parámetros en MLFlow
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("n_features", X_train.shape[1])
    mlflow.log_param("n_samples_train", X_train.shape[0])
    
    # Registrar métricas de entrenamiento en MLFlow
    mlflow.log_metric("train_rmse", train_metrics['RMSE'])
    mlflow.log_metric("train_mae", train_metrics['MAE'])
    mlflow.log_metric("train_r2", train_metrics['R2'])
    
    # Registrar métricas de prueba en MLFlow
    mlflow.log_metric("test_rmse", test_metrics['RMSE'])
    mlflow.log_metric("test_mae", test_metrics['MAE'])
    mlflow.log_metric("test_r2", test_metrics['R2'])
    
    # Guardar modelo en MLFlow
    mlflow.sklearn.log_model(pipeline_lr, "model")
    
    print("Métricas y modelo registrados en MLFlow.")

Entrenando modelo: Regresión Lineal
Modelo entrenado.


2025/10/28 16:09:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Evaluación en conjunto de ENTRENAMIENTO:

Métricas para Regresión Lineal - Train:
  RMSE (Root Mean Squared Error): $2,889.84
  MAE  (Mean Absolute Error):     $1,729.63
  R²   (R-squared):               0.7933 (79.33%)

Evaluación en conjunto de PRUEBA:

Métricas para Regresión Lineal - Test:
  RMSE (Root Mean Squared Error): $2,818.62
  MAE  (Mean Absolute Error):     $1,718.15
  R²   (R-squared):               0.7770 (77.70%)



2025/10/28 16:09:16 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/28 16:09:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Métricas y modelo registrados en MLFlow.


## 7. Entrenamiento - Modelo 2: Random Forest

Random Forest es un modelo basado en árboles de decisión que:
- Puede capturar relaciones no lineales
- Es robusto a outliers
- Maneja bien la interacción entre variables
- Generalmente tiene mejor desempeño que regresión lineal en datos complejos

In [60]:
# Iniciar un nuevo run de MLFlow para Random Forest
with mlflow.start_run(run_name="random_forest"):
    
    print("Entrenando modelo: Random Forest")
    
    # Definir hiperparámetros para Random Forest
    rf_params = {
        'n_estimators': 100,        # Número de árboles en el bosque
        'max_depth': 20,            # Profundidad máxima de cada árbol
        'min_samples_split': 5,     # Mínimo de muestras para dividir un nodo
        'min_samples_leaf': 2,      # Mínimo de muestras en una hoja
        'random_state': 42,         # Para reproducibilidad
        'n_jobs': -1                # Usar todos los cores disponibles
    }
    
    # Crear pipeline completo
    pipeline_rf = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(**rf_params))
    ])
    
    # Entrenar modelo
    pipeline_rf.fit(X_train, y_train)
    print("Modelo entrenado.")
    
    # Hacer predicciones
    y_train_pred_rf = pipeline_rf.predict(X_train)
    y_test_pred_rf = pipeline_rf.predict(X_test)
    
    # Evaluar en conjunto de entrenamiento
    print("\nEvaluación en conjunto de ENTRENAMIENTO:")
    train_metrics_rf = evaluate_model(y_train, y_train_pred_rf, "Random Forest - Train")
    
    # Evaluar en conjunto de prueba
    print("Evaluación en conjunto de PRUEBA:")
    test_metrics_rf = evaluate_model(y_test, y_test_pred_rf, "Random Forest - Test")
    
    # Registrar parámetros en MLFlow
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("n_features", X_train.shape[1])
    mlflow.log_param("n_samples_train", X_train.shape[0])
    
    # Registrar hiperparámetros específicos de Random Forest
    for param_name, param_value in rf_params.items():
        mlflow.log_param(f"rf_{param_name}", param_value)
    
    # Registrar métricas de entrenamiento en MLFlow
    mlflow.log_metric("train_rmse", train_metrics_rf['RMSE'])
    mlflow.log_metric("train_mae", train_metrics_rf['MAE'])
    mlflow.log_metric("train_r2", train_metrics_rf['R2'])
    
    # Registrar métricas de prueba en MLFlow
    mlflow.log_metric("test_rmse", test_metrics_rf['RMSE'])
    mlflow.log_metric("test_mae", test_metrics_rf['MAE'])
    mlflow.log_metric("test_r2", test_metrics_rf['R2'])
    
    input_example = X_train.iloc[:5]

    # Guardar modelo en MLFlow
    mlflow.sklearn.log_model(
        pipeline_rf, 
        name="model",
        input_example=input_example
    )
    
    print("Métricas y modelo registrados en MLFlow.")

Entrenando modelo: Random Forest
Modelo entrenado.

Evaluación en conjunto de ENTRENAMIENTO:

Métricas para Random Forest - Train:
  RMSE (Root Mean Squared Error): $649.88
  MAE  (Mean Absolute Error):     $301.62
  R²   (R-squared):               0.9895 (98.95%)

Evaluación en conjunto de PRUEBA:

Métricas para Random Forest - Test:
  RMSE (Root Mean Squared Error): $1,309.54
  MAE  (Mean Absolute Error):     $632.21
  R²   (R-squared):               0.9519 (95.19%)



2025/10/28 16:09:18 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Métricas y modelo registrados en MLFlow.


## 7.5. Entrenamiento - Modelo 3: K-Nearest Neighbors

K-Nearest Neighbors (KNN) es un modelo basado en instancias que:
- Predice basándose en los K vecinos más cercanos
- No requiere un proceso de entrenamiento tradicional
- Puede capturar patrones locales complejos
- Es sensible a la escala de las características (por eso usamos StandardScaler)

In [61]:
# Iniciar un nuevo run de MLFlow para KNN
with mlflow.start_run(run_name="knn_regressor"):
    
    print("Entrenando modelo: K-Nearest Neighbors")
    
    # Definir hiperparámetros para KNN
    knn_params = {
        'n_neighbors': 5,       # Número de vecinos a considerar
        'weights': 'distance',  # Ponderar por distancia inversa
        'metric': 'euclidean',  # Métrica de distancia
        'n_jobs': -1           # Usar todos los cores disponibles
    }
    
    # Crear pipeline completo
    pipeline_knn = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', KNeighborsRegressor(**knn_params))
    ])
    
    # Entrenar modelo
    pipeline_knn.fit(X_train, y_train)
    print("Modelo entrenado.")
    
    # Hacer predicciones
    y_train_pred_knn = pipeline_knn.predict(X_train)
    y_test_pred_knn = pipeline_knn.predict(X_test)
    
    # Evaluar en conjunto de entrenamiento
    print("\nEvaluación en conjunto de ENTRENAMIENTO:")
    train_metrics_knn = evaluate_model(y_train, y_train_pred_knn, "KNN - Train")
    
    # Evaluar en conjunto de prueba
    print("Evaluación en conjunto de PRUEBA:")
    test_metrics_knn = evaluate_model(y_test, y_test_pred_knn, "KNN - Test")
    
    # Registrar parámetros en MLFlow
    mlflow.log_param("model_type", "KNeighborsRegressor")
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("n_features", X_train.shape[1])
    mlflow.log_param("n_samples_train", X_train.shape[0])
    
    # Registrar hiperparámetros específicos de KNN
    for param_name, param_value in knn_params.items():
        mlflow.log_param(f"knn_{param_name}", param_value)
    
    # Registrar métricas de entrenamiento en MLFlow
    mlflow.log_metric("train_rmse", train_metrics_knn['RMSE'])
    mlflow.log_metric("train_mae", train_metrics_knn['MAE'])
    mlflow.log_metric("train_r2", train_metrics_knn['R2'])
    
    # Registrar métricas de prueba en MLFlow
    mlflow.log_metric("test_rmse", test_metrics_knn['RMSE'])
    mlflow.log_metric("test_mae", test_metrics_knn['MAE'])
    mlflow.log_metric("test_r2", test_metrics_knn['R2'])
    
    # Guardar modelo en MLFlow
    mlflow.sklearn.log_model(pipeline_knn, "model")
    
    print("Métricas y modelo registrados en MLFlow.")

2025/10/28 16:09:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Entrenando modelo: K-Nearest Neighbors
Modelo entrenado.

Evaluación en conjunto de ENTRENAMIENTO:

Métricas para KNN - Train:
  RMSE (Root Mean Squared Error): $0.00
  MAE  (Mean Absolute Error):     $0.00
  R²   (R-squared):               1.0000 (100.00%)

Evaluación en conjunto de PRUEBA:

Métricas para KNN - Test:
  RMSE (Root Mean Squared Error): $3,180.79
  MAE  (Mean Absolute Error):     $1,693.64
  R²   (R-squared):               0.7160 (71.60%)



2025/10/28 16:09:20 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/28 16:09:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Métricas y modelo registrados en MLFlow.


## 7.75. Entrenamiento - Modelo 4: LightGBM

LightGBM (Light Gradient Boosting Machine) es un framework de gradient boosting que:
- Es extremadamente eficiente en tiempo y memoria
- Maneja grandes volúmenes de datos
- Tiene excelente desempeño predictivo
- Soporta datos categóricos nativamente

In [62]:
# Iniciar un nuevo run de MLFlow para LightGBM
with mlflow.start_run(run_name="lightgbm"):
    
    print("Entrenando modelo: LightGBM")
    
    # Definir hiperparámetros para LightGBM
    lgb_params = {
        'n_estimators': 100,
        'learning_rate': 0.1,
        'max_depth': 20,
        'num_leaves': 31,
        'random_state': 42,
        'n_jobs': -1,
        'verbose': -1  # Silenciar warnings
    }
    
    # Crear pipeline completo
    pipeline_lgb = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', lgb.LGBMRegressor(**lgb_params))
    ])
    
    # Entrenar modelo
    pipeline_lgb.fit(X_train, y_train)
    print("Modelo entrenado.")
    
    # Hacer predicciones
    y_train_pred_lgb = pipeline_lgb.predict(X_train)
    y_test_pred_lgb = pipeline_lgb.predict(X_test)
    
    # Evaluar en conjunto de entrenamiento
    print("\nEvaluación en conjunto de ENTRENAMIENTO:")
    train_metrics_lgb = evaluate_model(y_train, y_train_pred_lgb, "LightGBM - Train")
    
    # Evaluar en conjunto de prueba
    print("Evaluación en conjunto de PRUEBA:")
    test_metrics_lgb = evaluate_model(y_test, y_test_pred_lgb, "LightGBM - Test")
    
    # Registrar parámetros en MLFlow
    mlflow.log_param("model_type", "LightGBM")
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("n_features", X_train.shape[1])
    mlflow.log_param("n_samples_train", X_train.shape[0])
    
    # Registrar hiperparámetros específicos de LightGBM
    for param_name, param_value in lgb_params.items():
        mlflow.log_param(f"lgb_{param_name}", param_value)
    
    # Registrar métricas de entrenamiento en MLFlow
    mlflow.log_metric("train_rmse", train_metrics_lgb['RMSE'])
    mlflow.log_metric("train_mae", train_metrics_lgb['MAE'])
    mlflow.log_metric("train_r2", train_metrics_lgb['R2'])
    
    # Registrar métricas de prueba en MLFlow
    mlflow.log_metric("test_rmse", test_metrics_lgb['RMSE'])
    mlflow.log_metric("test_mae", test_metrics_lgb['MAE'])
    mlflow.log_metric("test_r2", test_metrics_lgb['R2'])
    
    # Guardar modelo en MLFlow
    mlflow.sklearn.log_model(pipeline_lgb, "model")
    
    print("Métricas y modelo registrados en MLFlow.")

Entrenando modelo: LightGBM


2025/10/28 16:09:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Modelo entrenado.

Evaluación en conjunto de ENTRENAMIENTO:

Métricas para LightGBM - Train:
  RMSE (Root Mean Squared Error): $582.63
  MAE  (Mean Absolute Error):     $310.53
  R²   (R-squared):               0.9916 (99.16%)

Evaluación en conjunto de PRUEBA:

Métricas para LightGBM - Test:
  RMSE (Root Mean Squared Error): $915.82
  MAE  (Mean Absolute Error):     $405.68
  R²   (R-squared):               0.9765 (97.65%)



2025/10/28 16:09:23 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/28 16:09:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Métricas y modelo registrados en MLFlow.


## 8. Comparación de Modelos

In [63]:
# Crear tabla comparativa con todos los modelos
comparison_df = pd.DataFrame({
    'Modelo': ['Regresión Lineal', 'Random Forest', 'KNN', 'LightGBM'],
    'RMSE (Test)': [test_metrics['RMSE'], test_metrics_rf['RMSE'], test_metrics_knn['RMSE'], test_metrics_lgb['RMSE']],
    'MAE (Test)': [test_metrics['MAE'], test_metrics_rf['MAE'], test_metrics_knn['MAE'], test_metrics_lgb['MAE']],
    'R² (Test)': [test_metrics['R2'], test_metrics_rf['R2'], test_metrics_knn['R2'], test_metrics_lgb['R2']],
    'R² (Train)': [train_metrics['R2'], train_metrics_rf['R2'], train_metrics_knn['R2'], train_metrics_lgb['R2']]
})

# Calcular overfitting (diferencia entre R² train y test)
comparison_df['Overfitting (R²)'] = comparison_df['R² (Train)'] - comparison_df['R² (Test)']

print("\n" + "="*80)
print("COMPARACIÓN DE MODELOS")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Identificar mejor modelo (basado en R² de test)
best_model_idx = comparison_df['R² (Test)'].idxmax()
best_model_name = comparison_df.loc[best_model_idx, 'Modelo']
best_r2 = comparison_df.loc[best_model_idx, 'R² (Test)']

print(f"\nMejor modelo: {best_model_name} (R² = {best_r2:.4f})")


COMPARACIÓN DE MODELOS
          Modelo  RMSE (Test)  MAE (Test)  R² (Test)  R² (Train)  Overfitting (R²)
Regresión Lineal  2818.615068 1718.152500   0.776961    0.793335          0.016374
   Random Forest  1309.536472  632.206624   0.951856    0.989548          0.037692
             KNN  3180.793002 1693.636244   0.715960    1.000000          0.284040
        LightGBM   915.820624  405.684479   0.976453    0.991599          0.015146

Mejor modelo: LightGBM (R² = 0.9765)


## 9. Análisis de Importancia de Variables (Random Forest)

Random Forest permite identificar qué variables contribuyen más a las predicciones.

In [64]:
# Obtener el modelo Random Forest del pipeline
rf_model = pipeline_rf.named_steps['regressor']

# Obtener nombres de características después del preprocesamiento
# Para categóricas, OneHotEncoder crea múltiples columnas
cat_encoder = pipeline_rf.named_steps['preprocessor'].named_transformers_['cat']
cat_feature_names = cat_encoder.get_feature_names_out(categorical_features)
feature_names = list(numeric_features) + list(cat_feature_names)

# Crear DataFrame con importancias
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop 15 variables más importantes (Random Forest):")
print(feature_importance_df.head(15).to_string(index=False))


Top 15 variables más importantes (Random Forest):
                  Feature  Importance
                      año    0.534450
      calificacion_estado    0.169433
      marca_Mercedes-Benz    0.074182
               marca_Audi    0.067488
                marca_BMW    0.051460
              kilometraje    0.041028
      tipo_carroceria_SUV    0.013850
                  consumo    0.011631
tipo_carroceria_Hatchback    0.007182
                     peso    0.003702
    tipo_carroceria_Sedán    0.003426
               cilindrada    0.003130
                 potencia    0.002930
   tipo_carroceria_Pickup    0.002014
tipo_combustible_Gasolina    0.001769


## 10. Guardar Mejor Modelo

Guardamos el modelo con mejor desempeño para su uso posterior en producción.

In [ ]:
# Crear directorio para modelos
models_dir = Path('../models')
models_dir.mkdir(exist_ok=True)

# Determinar qué modelo guardar (el de mejor R²)
model_mapping = {
    'Regresión Lineal': (pipeline_lr, 'linear_regression_best_model.pkl'),
    'Random Forest': (pipeline_rf, 'random_forest_best_model.pkl'),
    'KNN': (pipeline_knn, 'knn_best_model.pkl'),
    'LightGBM': (pipeline_lgb, 'lightgbm_best_model.pkl')
}

best_pipeline, model_filename = model_mapping[best_model_name]

# Guardar modelo usando joblib (formato eficiente para sklearn)
model_path = models_dir / model_filename
joblib.dump(best_pipeline, model_path)

print(f"\nModelo guardado en: {model_path}")
print(f"Modelo: {best_model_name}")
print(f"R² en test: {best_r2:.4f}")

# También guardamos información de las características
metadata = {
    'model_type': best_model_name,
    'model_file': model_filename,  # Agregar nombre del archivo del modelo
    'numeric_features': numeric_features,
    'categorical_features': categorical_features,
    'test_r2': float(best_r2),
    'feature_names': feature_names
}

import json
metadata_path = models_dir / 'model_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"Metadata guardada en: {metadata_path}")

## 11. Cómo Visualizar Experimentos en MLFlow UI

MLFlow UI te permite comparar modelos, visualizar métricas y descargar modelos de forma interactiva.

### 🚀 Opción 1: Script Automático (Recomendado)

Desde el directorio del proyecto:

```bash
cd prediccion-precios-automoviles
./start_mlflow.sh
```

El script detecta automáticamente:
- ✅ Si el puerto 5000 está ocupado
- ✅ La ubicación correcta de mlruns/
- ✅ Ofrece detener procesos anteriores

### 🔧 Opción 2: Comando Manual

Desde el directorio del proyecto:

```bash
cd prediccion-precios-automoviles
mlflow ui
```

MLFlow detectará automáticamente el directorio `mlruns/` en la ubicación actual.

### 🌐 Acceder a la UI

Abre tu navegador en: **http://localhost:5000**

### 📊 En la UI podrás:

- ✅ Comparar métricas entre diferentes runs (Regresión Lineal, Random Forest, KNN, LightGBM)
- ✅ Visualizar parámetros e hiperparámetros de cada experimento
- ✅ Descargar modelos guardados
- ✅ Crear gráficos de comparación interactivos
- ✅ Buscar y filtrar experimentos por métricas
- ✅ Ver diferencias entre modelos en tiempo real

### ⚠️ Solución de Problemas

**Error: "Address already in use"**
- Otro proceso está usando el puerto 5000
- Solución: Ejecuta `./start_mlflow.sh` (detecta y resuelve automáticamente)
- O manualmente: `pkill -f "mlflow ui"` y luego `mlflow ui`

**Nota:** Este proyecto está configurado para ser **completamente portable**. No necesitas modificar rutas, funciona igual para todos los usuarios.

## Conclusión

### Resultados del entrenamiento:

Se entrenaron exitosamente cuatro modelos de regresión y se registraron sus métricas en MLFlow:

1. **Regresión Lineal** (modelo baseline):
   - Modelo simple que asume relaciones lineales
   - R² = 77.70% en test
   - Útil como punto de referencia
   - Interpretabilidad alta

2. **Random Forest** (modelo ensemble):
   - Captura relaciones no lineales y complejas
   - R² = 95.19% en test
   - Proporciona importancia de variables
   - Más robusto a outliers

3. **K-Nearest Neighbors** (modelo basado en instancias):
   - Basado en similitud entre muestras
   - R² = 71.60% en test
   - Presenta overfitting significativo (100% train vs 71.60% test)
   - Sensible a la escala de las características

4. **LightGBM** (modelo ganador):
   - Framework de gradient boosting eficiente
   - **R² = 97.65% en test** (mejor desempeño)
   - Excelente balance entre precisión y generalización
   - Bajo overfitting (99.16% train vs 97.65% test)

### Modelo seleccionado:

**LightGBM** fue seleccionado como el mejor modelo por:
- Mayor R² en conjunto de prueba (97.65%)
- Mejor generalización (menor overfitting)
- Excelente desempeño en todas las métricas
- Eficiencia computacional

### Próximos pasos:

Con el mejor modelo guardado, podemos:
1. Implementar módulos Python reutilizables (config, preprocessing, training, predict)
2. Crear interfaces de usuario con Gradio
3. Desarrollar APIs con FastAPI para servir predicciones
4. Realizar fine-tuning de hiperparámetros con validación cruzada
5. Experimentar con otros modelos avanzados (XGBoost, CatBoost)